In [17]:
import numpy as np 
import pandas as pd
import re
import string
import gensim
import pickle
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk import punkt
from nltk import wordnet

from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout, GRU, Bidirectional, Conv1D, GlobalMaxPooling1D
from keras.utils.np_utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [18]:
data_columns = ["target", "tweet", "username"]
data_encoding = "ISO-8859-1"
data=pd.read_csv('./TwitterDataset.csv',encoding = data_encoding, usecols=data_columns)

X = data.iloc[:, 1]
Y = data.iloc[:, 0]

print(X.head())
print(data['target'].value_counts())


0    the real reason why you're sad? you're attache...
1        my biggest problem is overthinking everything
2    the worst sadness is the sadness you've taught...
3    i cannot make you understand. i cannot make an...
4    i don't think anyone really understands how ti...
Name: tweet, dtype: object
0    4695
1    3440
Name: target, dtype: int64


In [19]:
def preprocess(tweet):
    username = "@\S+"
    new_tweet = re.sub(username, ' ',tweet) # Remove @tags
    
    new_tweet = new_tweet.lower() # Smart lowercase
    
    new_tweet = re.sub(r'\d+', ' ', new_tweet) # Remove numbers
    
    text_noise = "https?:\S+|http?:\S|[^A-Za-z0-9]+" 
    new_tweet = re.sub(text_noise, ' ', new_tweet) # Remove links
    
    new_tweet = new_tweet.translate(new_tweet.maketrans('','',string.punctuation)) # Remove Punctuation
    
    new_tweet = new_tweet.strip() # Remove white spaces
    
    new_tweet = word_tokenize(new_tweet) # Tokenize into words
    
    new_tweet = ' '.join([word for word in new_tweet if word.isalpha()]) # Remove non alphabetic tokens
    
    stop_words = set(stopwords.words('english'))
    new_tweet = ' '.join([word for word in new_tweet.split() if not word in stop_words]) # Filter out stop words
    
    lemmatizer = WordNetLemmatizer()
    new_tweet = ' '.join([lemmatizer.lemmatize(word,"v") for word in new_tweet.split()]) # Word Lemmatization
    
    return new_tweet

In [20]:
X = X.apply(preprocess)

print(X)

0       real reason sad attach people distant pay atte...
1                 biggest problem overthinking everything
2                        worst sadness sadness teach hide
3       make understand make anyone understand happen ...
4       think anyone really understand tire act okay a...
                              ...                        
8130    cardi b want trademark catchphrase okurr think...
8131    bet kellyanne george conway pretty disturb mak...
8132    fan always ask watch old stuff finally answer ...
8133    ray romano hilarious comedian kind soul rare n...
8134    mueller report may finish mine next week johnn...
Name: tweet, Length: 8135, dtype: object


In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print("TRAIN size:", len(X_train))
print("TEST size:", len(X_test))

TRAIN size: 6508
TEST size: 1627


In [22]:
w2v_size = 300
w2v_win = 7
w2v_epoch = 32
w2v_mincount = 10

document = [tweet.split() for tweet in X_train]
word2vec_model = gensim.models.word2vec.Word2Vec(vector_size=w2v_size,
                                                window=w2v_win,
                                                min_count=w2v_mincount,
                                                workers=8)

word2vec_model.build_vocab(document)

In [23]:
words = word2vec_model.wv.index_to_key
vocabulary_size = len(words)
print('Vocabulary_size ::: ',vocabulary_size)

Vocabulary_size :::  964


In [24]:
word2vec_model.train(document,total_examples=len(document),epochs=w2v_epoch)

(1186002, 1851840)

In [25]:
max_sequence_length = 300
vector_size = 300

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
unique_tokens = tokenizer.word_index
tokens_size = len(unique_tokens)
print('No.of unique tokens === %s'%tokens_size)

No.of unique tokens === 8635


In [26]:
X_train_padded = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_padded,maxlen=max_sequence_length)

In [27]:
vector_matrix = np.zeros((tokens_size+1,w2v_size))

for word,i in tokenizer.word_index.items():
    if word in word2vec_model.wv:
        vector_matrix[i]=word2vec_model.wv[word]

print('vector matrix shape === ',vector_matrix.shape)

vector_matrix

vector matrix shape ===  (8636, 300)


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.1113467 ,  0.29305074, -0.08593185, ...,  0.04234217,
         0.066323  , -0.03904191],
       [-0.48540571,  0.01416034, -0.364999  , ..., -0.54108834,
         0.42218277, -0.51971227],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [28]:
X_test_padded = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(X_test_padded,maxlen=max_sequence_length)

In [49]:
model = Sequential()
model.add(Embedding(tokens_size+1,w2v_size, weights=[vector_matrix],input_length=max_sequence_length,trainable=False))
model.add(Dropout(0.5))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_18 (Embedding)    (None, 300, 300)          2590800   
                                                                 
 dropout_18 (Dropout)        (None, 300, 300)          0         
                                                                 
 lstm_17 (LSTM)              (None, 100)               160400    
                                                                 
 dense_17 (Dense)            (None, 1)                 101       
                                                                 
Total params: 2,751,301
Trainable params: 160,501
Non-trainable params: 2,590,800
_________________________________________________________________


In [37]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]


In [38]:
%%time
history = model.fit(X_train_padded, Y_train,
                    batch_size=128,
                    epochs=50,
                    validation_split=0.1,
                    verbose=1,
                    callbacks=callbacks)

Epoch 1/50
46/46 [==============================] - 53s 1s/step - loss: 0.4146 - accuracy: 0.8084 - val_loss: 0.3448 - val_accuracy: 0.8602 - lr: 0.0010
Epoch 2/50
46/46 [==============================] - 52s 1s/step - loss: 0.3342 - accuracy: 0.8494 - val_loss: 0.3079 - val_accuracy: 0.8694 - lr: 0.0010
Epoch 3/50
46/46 [==============================] - 50s 1s/step - loss: 0.3220 - accuracy: 0.8549 - val_loss: 0.3075 - val_accuracy: 0.8618 - lr: 0.0010
Epoch 4/50
46/46 [==============================] - 50s 1s/step - loss: 0.3149 - accuracy: 0.8622 - val_loss: 0.3095 - val_accuracy: 0.8618 - lr: 0.0010
Epoch 5/50
46/46 [==============================] - 52s 1s/step - loss: 0.3098 - accuracy: 0.8663 - val_loss: 0.2838 - val_accuracy: 0.8817 - lr: 0.0010
Epoch 6/50
46/46 [==============================] - 51s 1s/step - loss: 0.3041 - accuracy: 0.8675 - val_loss: 0.2764 - val_accuracy: 0.8848 - lr: 0.0010
Epoch 7/50
46/46 [==============================] - 50s 1s/step - loss: 0.3022 - a

46/46 [==============================] - 52s 1s/step - loss: 0.2276 - accuracy: 0.9032 - val_loss: 0.2690 - val_accuracy: 0.8955 - lr: 0.0010
Epoch 23/50
46/46 [==============================] - 50s 1s/step - loss: 0.2217 - accuracy: 0.9040 - val_loss: 0.2747 - val_accuracy: 0.8955 - lr: 0.0010
Epoch 24/50
46/46 [==============================] - 52s 1s/step - loss: 0.2157 - accuracy: 0.9071 - val_loss: 0.2845 - val_accuracy: 0.8848 - lr: 0.0010
Epoch 25/50
46/46 [==============================] - 50s 1s/step - loss: 0.2072 - accuracy: 0.9087 - val_loss: 0.2680 - val_accuracy: 0.8986 - lr: 1.0000e-04
Epoch 26/50
46/46 [==============================] - 49s 1s/step - loss: 0.2036 - accuracy: 0.9117 - val_loss: 0.2679 - val_accuracy: 0.9002 - lr: 1.0000e-04
Epoch 27/50
46/46 [==============================] - 51s 1s/step - loss: 0.2052 - accuracy: 0.9099 - val_loss: 0.2692 - val_accuracy: 0.8986 - lr: 1.0000e-04
Epoch 28/50
46/46 [==============================] - 49s 1s/step - loss: 0.2

Epoch 43/50
46/46 [==============================] - 51s 1s/step - loss: 0.2024 - accuracy: 0.9081 - val_loss: 0.2695 - val_accuracy: 0.8986 - lr: 1.0000e-07
Epoch 44/50
46/46 [==============================] - 50s 1s/step - loss: 0.2027 - accuracy: 0.9141 - val_loss: 0.2695 - val_accuracy: 0.8986 - lr: 1.0000e-07
Epoch 45/50
46/46 [==============================] - 50s 1s/step - loss: 0.2005 - accuracy: 0.9126 - val_loss: 0.2695 - val_accuracy: 0.8986 - lr: 1.0000e-08
Epoch 46/50
46/46 [==============================] - 51s 1s/step - loss: 0.1990 - accuracy: 0.9139 - val_loss: 0.2695 - val_accuracy: 0.8986 - lr: 1.0000e-08
Epoch 47/50
46/46 [==============================] - 50s 1s/step - loss: 0.2032 - accuracy: 0.9104 - val_loss: 0.2695 - val_accuracy: 0.8986 - lr: 1.0000e-08
Epoch 48/50
46/46 [==============================] - 50s 1s/step - loss: 0.1987 - accuracy: 0.9129 - val_loss: 0.2695 - val_accuracy: 0.8986 - lr: 1.0000e-08
Epoch 49/50
46/46 [==============================] -

In [39]:
#best_model=rand_search.best_estimator_.model
display(model.evaluate(X_train_padded,Y_train), 'train')
display(model.evaluate(X_test_padded, Y_test), 'test')

204/204 [==============================] - 37s 180ms/step - loss: 0.1855 - accuracy: 0.9218


[0.18548548221588135, 0.9217885732650757]

'train'

51/51 [==============================] - 9s 177ms/step - loss: 0.3739 - accuracy: 0.8347


[0.37394848465919495, 0.8346650004386902]

'test'

In [40]:
from sklearn.metrics import classification_report

predictions = model.predict(X_train_padded)
predictions = [1 if p >= 0.5 else 0 for p in predictions]

print(classification_report(Y_train, predictions, target_names=['not depressed', 'depressed']))

               precision    recall  f1-score   support

not depressed       0.94      0.92      0.93      3777
    depressed       0.90      0.92      0.91      2731

     accuracy                           0.92      6508
    macro avg       0.92      0.92      0.92      6508
 weighted avg       0.92      0.92      0.92      6508



In [41]:
predictions = model.predict(X_test_padded)
predictions = [1 if p >= 0.5 else 0 for p in predictions]

print(classification_report(Y_test, predictions, target_names=['not depressed', 'depressed']))

               precision    recall  f1-score   support

not depressed       0.87      0.84      0.85       918
    depressed       0.80      0.83      0.81       709

     accuracy                           0.83      1627
    macro avg       0.83      0.83      0.83      1627
 weighted avg       0.84      0.83      0.83      1627



In [43]:
model_json = model.to_json()
with open('LSTM_Model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('LSTM_weights.h5')

In [45]:
from keras.layers.convolutional import MaxPooling1D

In [48]:
model2 = Sequential()
model2.add(Embedding(tokens_size+1,w2v_size, weights=[vector_matrix],input_length=max_sequence_length,trainable=False))
model2.add(Dropout(0.5))
model2.add(Conv1D(filters=300, kernel_size=3, padding='same', activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
model2.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(1, activation='sigmoid'))

model2.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, 300, 300)          2590800   
                                                                 
 dropout_17 (Dropout)        (None, 300, 300)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 300, 300)          270300    
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 150, 300)         0         
 1D)                                                             
                                                                 
 lstm_16 (LSTM)              (None, 100)               160400    
                                                                 
 dense_16 (Dense)            (None, 1)                 101       
                                                     

In [50]:
model2.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]


In [52]:
%%time
history2 = model2.fit(X_train_padded, Y_train,
                    batch_size=128,
                    epochs=80,
                    validation_split=0.1,
                    verbose=1,
                    callbacks=callbacks)

Epoch 1/80
46/46 [==============================] - 46s 983ms/step - loss: 0.3858 - accuracy: 0.8223 - val_loss: 0.3136 - val_accuracy: 0.8633 - lr: 0.0010
Epoch 2/80
46/46 [==============================] - 45s 986ms/step - loss: 0.3225 - accuracy: 0.8614 - val_loss: 0.3071 - val_accuracy: 0.8648 - lr: 0.0010
Epoch 3/80
46/46 [==============================] - 46s 993ms/step - loss: 0.3097 - accuracy: 0.8658 - val_loss: 0.3059 - val_accuracy: 0.8633 - lr: 0.0010
Epoch 4/80
46/46 [==============================] - 45s 982ms/step - loss: 0.2973 - accuracy: 0.8687 - val_loss: 0.3030 - val_accuracy: 0.8556 - lr: 0.0010
Epoch 5/80
46/46 [==============================] - 46s 995ms/step - loss: 0.2896 - accuracy: 0.8730 - val_loss: 0.3034 - val_accuracy: 0.8633 - lr: 0.0010
Epoch 6/80
46/46 [==============================] - 46s 994ms/step - loss: 0.2813 - accuracy: 0.8762 - val_loss: 0.2965 - val_accuracy: 0.8679 - lr: 0.0010
Epoch 7/80
46/46 [==============================] - 45s 983ms/st

Epoch 22/80
46/46 [==============================] - 28s 619ms/step - loss: 0.1459 - accuracy: 0.9414 - val_loss: 0.2837 - val_accuracy: 0.8894 - lr: 1.0000e-05
Epoch 23/80
46/46 [==============================] - 29s 621ms/step - loss: 0.1451 - accuracy: 0.9404 - val_loss: 0.2829 - val_accuracy: 0.8909 - lr: 1.0000e-05
Epoch 24/80
46/46 [==============================] - 29s 639ms/step - loss: 0.1481 - accuracy: 0.9401 - val_loss: 0.2822 - val_accuracy: 0.8894 - lr: 1.0000e-05
Epoch 25/80
46/46 [==============================] - 30s 644ms/step - loss: 0.1485 - accuracy: 0.9384 - val_loss: 0.2820 - val_accuracy: 0.8894 - lr: 1.0000e-05
Epoch 26/80
46/46 [==============================] - 29s 635ms/step - loss: 0.1431 - accuracy: 0.9401 - val_loss: 0.2822 - val_accuracy: 0.8894 - lr: 1.0000e-06
Epoch 27/80
46/46 [==============================] - 30s 643ms/step - loss: 0.1497 - accuracy: 0.9392 - val_loss: 0.2825 - val_accuracy: 0.8909 - lr: 1.0000e-06
Epoch 28/80
46/46 [===============

Epoch 43/80
46/46 [==============================] - 28s 608ms/step - loss: 0.1434 - accuracy: 0.9409 - val_loss: 0.2827 - val_accuracy: 0.8909 - lr: 1.0000e-09
Epoch 44/80
46/46 [==============================] - 29s 630ms/step - loss: 0.1466 - accuracy: 0.9401 - val_loss: 0.2827 - val_accuracy: 0.8909 - lr: 1.0000e-09
Epoch 45/80
46/46 [==============================] - 29s 638ms/step - loss: 0.1467 - accuracy: 0.9397 - val_loss: 0.2827 - val_accuracy: 0.8909 - lr: 1.0000e-09
Epoch 46/80
46/46 [==============================] - 30s 653ms/step - loss: 0.1487 - accuracy: 0.9399 - val_loss: 0.2827 - val_accuracy: 0.8909 - lr: 1.0000e-10
Epoch 47/80
35/46 [=====================>........] - ETA: 6s - loss: 0.1443 - accuracy: 0.9406

KeyboardInterrupt: 

In [53]:
#best_model=rand_search.best_estimator_.model
display(model2.evaluate(X_train_padded,Y_train), 'train')
display(model2.evaluate(X_test_padded, Y_test), 'test')

204/204 [==============================] - 13s 63ms/step - loss: 0.1247 - accuracy: 0.9542


[0.12466292828321457, 0.9542102217674255]

'train'

51/51 [==============================] - 3s 64ms/step - loss: 0.4207 - accuracy: 0.8334


[0.42073795199394226, 0.8334357738494873]

'test'

In [55]:
from sklearn.metrics import classification_report

predictions = model2.predict(X_train_padded)
predictions = [1 if p >= 0.5 else 0 for p in predictions]

print(classification_report(Y_train, predictions, target_names=['not depressed', 'depressed']))

               precision    recall  f1-score   support

not depressed       0.97      0.95      0.96      3777
    depressed       0.93      0.96      0.95      2731

     accuracy                           0.95      6508
    macro avg       0.95      0.95      0.95      6508
 weighted avg       0.95      0.95      0.95      6508



In [56]:
predictions = model2.predict(X_test_padded)
predictions = [1 if p >= 0.5 else 0 for p in predictions]

print(classification_report(Y_test, predictions, target_names=['not depressed', 'depressed']))

               precision    recall  f1-score   support

not depressed       0.87      0.83      0.85       918
    depressed       0.79      0.84      0.81       709

     accuracy                           0.83      1627
    macro avg       0.83      0.83      0.83      1627
 weighted avg       0.84      0.83      0.83      1627



In [58]:
model_json = model2.to_json()
with open('LSTM_CNN_Model.json', 'w') as json_file:
    json_file.write(model_json)
model2.save_weights('LSTM_CNN_weights.h5')